#MobiAct_ADL_load_dataset.ipynb. 
Loads the 6 Activities of Daily living (ADL) from the first release of the MobiAct dataset and converts to numpy arrays while adhering to the general format of the [Keras MNIST load_data function](https://keras.io/api/datasets/mnist/#load_data-function).

STD Standing (1x5min)
WAL Walking (1x5min)
JOG Jogging (3x30sec)
JUM Jumping (3x30sec)
STU Stairs Up (6x10sec)
STN Stairs Down (6x10sec)

Only the timestamp 'nanoseconds' and accelerometer data (accel_x/y/z) is imported. Sitting, Fall, gyro, and orientation data is not used.

Data is segmented into 500 samples (~5s) using as much of the sample as possible (incomplete segments are discarded)

Returns: Tuple of Numpy arrays:   
(x_train, y_train),(x_valid, y_valid)\[optional\],(x_test, y_test) 

* x_train\/valid\/test: containing float64 with shapes (num_samples, 500, {3,4,1})
* y_train\/valid\/test: containing int8 with shapes (num_samples 0-9)

Default train/valid/test split is by subject with a best effort to keep gender mix and distributed by height among the three categories.
Split is 60%/20%/20%

Example usage:  
x_train, y_train, x_test, y_test = mobiact_adl_load_dataset()

Additional References:   
For more information on the dataset, please refer to the publication:    Vavoulas, G., Chatzaki, C., Malliotakis, T., Pediaditis, M. and Tsiknakis, M.,(2016) The MobiAct Dataset: Recognition of Activities of Daily Living using Smartphones.,In Proceedings of the International Conference on Information and Communication Technologies for Ageing Well and e-Health  – Volume 1: ICT4AWE, (ICT4AGEINGWELL 2016) ISBN 978-989-758-180-9, pages 143-151, Rome, Italy. DOI: 10.5220/0005792401430151

[tensorflow git repo with more datasets](https://github.com/tensorflow/datasets) which also links to another colab [Keras MNIST Example](https://colab.research.google.com/github/tensorflow/datasets/blob/master/docs/keras_example.ipynb) with more options.

Developed and tested using colab.research.google.com  
To save as .py version use File > Download .py

Author:  Lee B. Hinkle, IMICS Lab, Texas State University, 2021

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

TODOs:
* Still can't figure out how to download source directly from https://drive.google.com/file/d/0B5VcW5yHhWhibWxGRTZDd0dGY2s/edit.
* Implement one-hot encoding using libraries and also non-one-hot encode case with int
* Change dtypes to take up less space
* verbose needs to be global and cleaned up
* add gyro channels and implement list based channel selection versus bools

In [79]:
import os
import shutil #https://docs.python.org/3/library/shutil.html
from shutil import unpack_archive # to unzip
#from shutil import make_archive # to create zip for storage
import requests #for downloading zip file
import glob # to generate lists of files in directory - unix style pathnames
#from scipy import io #for loadmat, matlab conversion
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt # for plotting - pandas uses matplotlib
from tabulate import tabulate # for verbose tables
from tensorflow.keras.utils import to_categorical # for one-hot encoding

In [80]:
interactive = True # run example calls for each function as Jupyter Notebook

In [81]:
interactive = False # skip to run as interactive, executes main as .py

In [82]:
# Trying to figure out how to download automatically to eliminate drive mount
# gdown doesn't work even with confirm=t option - could be google
# limiting downloads from scripts (the link works fine when signed in)
# https://github.com/wkentaro/gdown/issues/43
# tried AnirudhJayant06 colab suggestion of
# !pip install --upgrade --no-cache-dir gdown
# !gdown "0B5VcW5yHhWhibWxGRTZDd0dGY2s&confirm=t" # Download MobiAct zipfile
# still no luck and search for alternative locations came up empty.

In [83]:
def get_mobiact():
    zip_fname = 'MobiAct_Dataset_v1.0.zip'
    if (not os.path.isfile(zip_fname)):
        print(zip_fname, 'not found.\nPlease go to',\
        'https://drive.google.com/uc?id=0B5VcW5yHhWhibWxGRTZDd0dGY2s')
        print('to download, then copy the file to current working directory')
        print('upload to colab takes ~5min and gives "not a zipfile" error until finished')
        quit() # to exit when run as python script
    elif (not os.path.isdir('MobiAct_Dataset')):
        print("Unzipping MobiAct Dataset")
        shutil.unpack_archive(zip_fname,'.','zip')
    else:
        print("Using existing archive")
if interactive:
    get_mobiact()

In [84]:
def get_mobiact_fname_mdata(path_in):
    """returns dataframe with filename and metadata from mobiact directory
    args: path_in is location of files e.g. JOG, JUM, etc directories
    returns: pandas dataframe with one row for each file.
    columns are full filename, activity, subject, group {unassigned, train, validate, test}"""
    df = pd.DataFrame() # new empty dataframe
    GRP = ['train','valid','test']
    ACT = ['JOG','JUM','STD','STN','STU','WAL']
    for i in ACT:    
        sub_path = path_in + i + '/'
        fname_in = i + '_acc_*.txt'
        print("Generating filenames ", fname_in," from ", sub_path," directory")
        # Build list of matching files in each directory, make into dataframe
        # extract and add metadata
        in_files = glob.glob(os.path.join(sub_path, fname_in)) # generates a list of all matching files
        temp = pd.DataFrame(in_files,dtype="string") # easier in pandas, make column of filenames
        temp.columns = ['fname'] # name the column
        temp['ACT'] = i # add column with activity
        temp['SUB'] = temp['fname'].str.extract(r'(\d+)') # uses pd method and regex get SUB from fname
        # both columns are type object, would be better to set but didn't work first time
        df = pd.concat([df,temp], ignore_index=True ) # concat needs two df, add regardless of index   
    # add column for valid, test groups by subject, others are marked train
    df["SUB"] = pd.to_numeric(df["SUB"]) #convert from object to INT32
    return df
if interactive:
    df_flist = get_mobiact_fname_mdata('MobiAct_Dataset/')
    display(df_flist.head())

In [85]:
def assign_group(df,split_subj):
    """returns dataframe with filename and metadata from mobiact directory
    args: dataframe with mobiact fname, ACT, SUB columns
    returns: dataframe with GRP = {train, test, validate} column added
    GRP assignment is based on list in split_subj
    """
    df['GRP'] = 'unassigned' # add column
    df.loc[df['SUB'].isin(split_subj['train_subj']),'GRP'] = 'train'
    df.loc[df['SUB'].isin(split_subj['valid_subj']),'GRP'] = 'valid'
    df.loc[df['SUB'].isin(split_subj['test_subj']),'GRP'] = 'test'
    return df
if interactive:
    split_subj = {'train_subj':[2,4,5,9,10,16,18,20,23,24,26,27,28,32,34,35,
                            36,38,42,45,46,47,48,49,50,51,52,53,54,57],
                'valid_subj':[3,6,8,11,12,22,37,40,43,56],
                'test_subj':[7,19,21,25,29,33,39,41,44,55]}
    df_flist = assign_group(df_flist, split_subj)
    display(df_flist.head())
    print('Use iloc to access a single element')
    print(df_flist['fname'].iloc[0])


In [86]:
def read_mobiact_file(full_filename):
    """returns dataframe from Mobiact txt file accel_xyz data, skips metadata, labels columns"""
    df = pd.read_csv(full_filename,skiprows=16, header=None) #skip 16 lines of metadata
    # json better? https://docs.python.org/3/library/json.html#module-json initial tries weren't successful
    df.columns = ["nanoseconds", "accel_x", "accel_y", "accel_z"]
    return df
if interactive:
    my_df = read_mobiact_file(df_flist['fname'].iloc[0]) # iloc value is row
    display(my_df.head())

In [87]:
def add_total_accel(df_in):
    """computes rms of accel_x/y/z data, removed 1g component, adds accel_total column"""
    # compute rms accel and remove 1g due to gravity, removes rotational dependency
    dfx = df_in.pow(2)[['accel_x','accel_y','accel_z']] #square each accel
    df_sum = dfx.sum(axis=1) #add sum of squares, new 1 col df
    df_in.loc[:,'accel_total'] = df_sum.pow(0.5)-9.8  # sqrt + remove 1g gravity component
    del dfx, df_sum
    return df_in
if interactive:
    my_df =  add_total_accel(my_df)
    display(my_df.head())

In [88]:
def to_abs_time(df):
    """subtracts the value of 1st nanosecond sample from all samples"""
    start_time = df.loc[0,'nanoseconds']
    df.loc[:,'nanoseconds'] = df.loc[:,'nanoseconds'] - start_time
    return df
if interactive:
    my_df =  to_abs_time(my_df)
    display(my_df.head())

In [89]:
def get_df_from_file(fname, start_discard=100, end_discard=100):
    """processes mobiact file fname and returns dataframe of accel values
    inputs: fname = full filemane
            start_discard = number of initial samples to delete (default = 200)
            end_discard = number of final samples to delete (default = 200)
    output: pandas dataframe of shape (samples, accel_total)"""
    df = read_mobiact_file(fname)
    df = add_total_accel(df)
    df = df.drop(columns=['nanoseconds']) # note this loses some time info - datetime better?
    df.drop(df.head(start_discard).index,inplace=True) # drop first rows
    df.drop(df.tail(end_discard).index,inplace=True) # drop last rows
    return df
if interactive:
    my_df = get_df_from_file(df_flist['fname'].iloc[0])
    display(my_df.head())

In [90]:
def split_df_npX(df, num_samples = 500):
    """converts dataframe (samples, total_accel) into 3D numpy array of shape
    (num_segments, num_samples, total_accel). e.g. a 1800 row dataframe will
    result in a (3,500,1) numpy array.   The samples that don't populate an
    entire segment are discarded and there is no overlap (sliding window)
    input:  dataframe with one files worth of data, index=samples, column=total_accel
            num_samples = number samples (rows) in each segment, default = 500
    ouput:  numpy array in trainX format shape = (segments, samples/segment, 1)"""
    temp = df.to_numpy() # easier to reshape and final form needed anyway
    if (temp.shape[0]//num_samples)==0:
        print ("WARNING:  File contains less than ",num_samples,"samples and is discarded")
    temp2 = temp[0:num_samples*(temp.shape[0]//num_samples)] # truncate to multiple of num_samples
    # df.shape[1] is the number of columns, which should be preserved as the third dimension
    tempX = temp2.reshape(-1,num_samples,df.shape[1]) # won't work with out truncation
    return tempX
if interactive:
    np_temp = split_df_npX(my_df)
    print("Array shape",np_temp.shape)
    with np.printoptions(precision=3, suppress=True): # make output shorter
        print(np_temp[:5]) # first 5 entries

In [91]:
def mobiact_adl_load_dataset(
    verbose = True,
    #Pass location of the original MobiAct zip file here.
    #Easiest way to find this in colab is:
    #  -mount your google drive with MobiAct zip file (link is above)
    #  -navigate to the file using File menu to left
    #  -right click on file and select 'copy path', paste in next line
    # orig_zipfile = '/content/drive/My Drive/Datasets/MobiAct_Dataset_v1.0.zip',
    incl_xyz_accel = False, #include component accel_x/y/z in ____X data
    incl_ttl_accel = True, #add rms value of accel_x/y/z in ____X data
    incl_val_group = False, #True => returns x/y_test, x/y_valid, x/y_train
                           #False => combine test & valid groups
    split_subj = {'train_subj':[2,4,5,9,10,16,18,20,23,24,26,27,28,32,34,35,
                                36,38,42,45,46,47,48,49,50,51,52,53,54,57],
                   'valid_subj':[3,6,8,11,12,22,37,40,43,56],
                   'test_subj':[7,19,21,25,29,33,39,41,44,55]},
    one_hot_encode = True
    ):

    """Loads and processes the MobiAct dataset accel channels for ADL activities.

    Args:
        verbose (bool): controls level of output (default is True)
        incl_xyz_accel (bool): returns the component acceleration (default is False)
        incl_ttl_accel (bool): returns the magnitude of accel vector minus 1g
        incl_val_group (bool): return includes x_valid and y_valid arrays (default is False)
        split_sub (dict): subject numbers assigned to train, valid, test groups
        one_hot_encode (bool): return one-hot-encoded y arrays

    Returns:
        trainX, trainy, validX, validy, testX, testy np arrays
    """

    get_mobiact()
    df_flist = get_mobiact_fname_mdata('MobiAct_Dataset/')
    df_flist = assign_group(df_flist,split_subj)
    #Note:  STU, STN files don't contain 900 samples so 200 discard start/finish + 500 time step doesn't work
    # Create zero'd np arrays otherwise accumulates when run more than once
    # should be a better way to do this
    # FIXME:  Third dimension must follow accel return values - in all three lines
    trainX = np.zeros(shape=(1,500,4)) #otherwise accumulates when run more than once
    trainy = np.zeros(shape=(1,6))
    validX = np.zeros(shape=(1,500,4))
    validy = np.zeros(shape=(1,6))
    testX = np.zeros(shape=(1,500,4))
    testy = np.zeros(shape=(1,6))
    for i in df_flist.index:
        if ((df_flist['GRP'][i])=='train'):
            #print ("processing train file", df_flist['fname'][i])
            df = get_df_from_file(df_flist['fname'][i])
            tempX = split_df_npX(df)
            trainX = np.vstack([trainX, tempX])
            tempy = np.zeros(shape=(tempX.shape[0],6)) # 6 is number of ACT
            #one hot encoding of ACT
            #Class indices are  {'JOG': 0, 'JUM': 1, 'STD': 2, 'STN': 3, 'STU': 4, 'WAL': 5}
            #to match the ones generated by Keras image import
            if ((df_flist['ACT'][i])=='JOG'): tempy[:,0]=1
            if ((df_flist['ACT'][i])=='JUM'): tempy[:,1]=1
            if ((df_flist['ACT'][i])=='STD'): tempy[:,2]=1
            if ((df_flist['ACT'][i])=='STN'): tempy[:,3]=1
            if ((df_flist['ACT'][i])=='STU'): tempy[:,4]=1
            if ((df_flist['ACT'][i])=='WAL'): tempy[:,5]=1
            trainy = np.vstack([trainy, tempy])
        if ((df_flist['GRP'][i])=='valid'):
            #print ("processing test file", df_flist['fname'][i])
            df = get_df_from_file(df_flist['fname'][i])
            tempX = split_df_npX(df)
            validX = np.vstack([validX, tempX])
            tempy = np.zeros(shape=(tempX.shape[0],6)) # 6 is number of ACT
            #one hot encoding of ACT
            if ((df_flist['ACT'][i])=='JOG'): tempy[:,0]=1
            if ((df_flist['ACT'][i])=='JUM'): tempy[:,1]=1
            if ((df_flist['ACT'][i])=='STD'): tempy[:,2]=1
            if ((df_flist['ACT'][i])=='STN'): tempy[:,3]=1
            if ((df_flist['ACT'][i])=='STU'): tempy[:,4]=1
            if ((df_flist['ACT'][i])=='WAL'): tempy[:,5]=1
            validy = np.vstack([validy, tempy])
        if ((df_flist['GRP'][i])=='test'):
            #print ("processing test file", df_flist['fname'][i])
            df = get_df_from_file(df_flist['fname'][i])
            tempX = split_df_npX(df)
            testX = np.vstack([testX, tempX])
            tempy = np.zeros(shape=(tempX.shape[0],6)) # 6 is number of ACT
            #one hot encoding of ACT
            if ((df_flist['ACT'][i])=='JOG'): tempy[:,0]=1
            if ((df_flist['ACT'][i])=='JUM'): tempy[:,1]=1
            if ((df_flist['ACT'][i])=='STD'): tempy[:,2]=1
            if ((df_flist['ACT'][i])=='STN'): tempy[:,3]=1
            if ((df_flist['ACT'][i])=='STU'): tempy[:,4]=1
            if ((df_flist['ACT'][i])=='WAL'): tempy[:,5]=1
            testy = np.vstack([testy, tempy])
    # delete first row placeholders
    trainX = np.delete(trainX, (0), axis=0) 
    trainy = np.delete(trainy, (0), axis=0)
    validX = np.delete(validX, (0), axis=0) 
    validy = np.delete(validy, (0), axis=0)
    testX = np.delete(testX, (0), axis=0)
    testy = np.delete(testy, (0), axis=0)

    # delete channels using numpy.delete(arr, obj, axis=None)
    # this should be replaced with channel list, also returns empty if both False
    if (incl_ttl_accel == False):
        trainX = np.delete(trainX, 3, axis = 2)
        validX = np.delete(validX, 3, axis = 2)
        testX = np.delete(testX, 3, axis = 2)
    if (incl_xyz_accel == False):
        trainX = np.delete(trainX, [0,1,2], axis = 2)
        validX = np.delete(validX, [0,1,2], axis = 2)
        testX = np.delete(testX, [0,1,2], axis = 2)
    if (incl_val_group):
        return trainX, trainy, validX, validy, testX, testy
    else:
        return np.concatenate((trainX, validX), axis=0),\
            np.concatenate((trainy, validy), axis=0),\
            testX, testy


In [92]:
if __name__ == "__main__":
    print("Downloading and processing MobiAct dataset, ADL Portion")
    x_train, y_train, x_test, y_test = mobiact_adl_load_dataset()
    print("\nMobiAct ADL returned arrays with default settings:")
    print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)
    print(80*'-')
    x_train, y_train, x_valid, y_valid, x_test, y_test = mobiact_adl_load_dataset(incl_xyz_accel = True, incl_val_group= True)
    print("\nMobiAct ADL returned arrays, incl_xyz_accel and incl_val_grp are True:")
    print("x_train shape ",x_train.shape," y_train shape ", y_train.shape)
    print("x_valid shape  ",x_valid.shape," y_valid shape  ",y_valid.shape)
    print("x_test shape  ",x_test.shape," y_test shape  ",y_test.shape)

Using existing archive
Generating filenames  JOG_acc_*.txt  from  MobiAct_Dataset/JOG/  directory
Generating filenames  JUM_acc_*.txt  from  MobiAct_Dataset/JUM/  directory
Generating filenames  STD_acc_*.txt  from  MobiAct_Dataset/STD/  directory
Generating filenames  STN_acc_*.txt  from  MobiAct_Dataset/STN/  directory
Generating filenames  STU_acc_*.txt  from  MobiAct_Dataset/STU/  directory
Generating filenames  WAL_acc_*.txt  from  MobiAct_Dataset/WAL/  directory

MobiAct ADL returned arrays with default settings:
x_train shape  (5587, 500, 1)  y_train shape  (5587, 6)
x_test shape   (1395, 500, 1)  y_test shape   (1395, 6)
--------------------------------------------------------------------------------
Using existing archive
Generating filenames  JOG_acc_*.txt  from  MobiAct_Dataset/JOG/  directory
Generating filenames  JUM_acc_*.txt  from  MobiAct_Dataset/JUM/  directory
Generating filenames  STD_acc_*.txt  from  MobiAct_Dataset/STD/  directory
Generating filenames  STN_acc_*.tx

In [93]:

if False: # mount drive and change to true to save files interactively
    import time
    #output_dir = '/content/drive/MyDrive/Processed_Datasets/MobiAct/All_Accel_Numpys'
    output_dir = '.' # if running locally or you want to download files manually
    if (os.path.isdir(output_dir)):
        #quick check for existing files, '.ipynb_checkpoints' file 
        #makes it more complicated to see if directory is empty
        if (not os.path.isfile(output_dir + '/x_train.npy')):
            summary = "MobiAct data\n"
            summary += "Saved to " + output_dir + "\n"
            summary += "Generated by MobiAct_ADL_load_dataset.ipynb\n"
            summary += " on " + time.strftime('%b-%d-%Y_%H%M', time.localtime())+'\n'
            summary += "this version for ???? work\n"
            info_fname = output_dir +'/'+'README.txt'
            full_info = summary
            print(full_info)

            with open(info_fname, "w") as file_object:
                file_object.write(full_info)

            if True:
                np.save(output_dir + '/'+'x_train.npy',x_train)
                np.save(output_dir + '/'+'y_train.npy',y_train)
                np.save(output_dir + '/'+'x_valid.npy',x_valid)
                np.save(output_dir + '/'+'y_valid.npy',y_valid)
                np.save(output_dir + '/'+'x_test.npy',x_test)
                np.save(output_dir + '/'+'y_test.npy',y_test)

        else:
            print("Error "+output_dir+" contains x_train.npy, please delete files")
    else:
        print(output_dir + " not found, please create directory") 